In [7]:
import requests

# ——— 定数設定 ———
CLIENT_ID = "dj00aiZpPXlkOGd5bDlUcTlWRyZzPWNvbnN1bWVyc2VjcmV0Jng9MmE-"
JAN_CODE = "622356249829"
API_ENDPOINT = "https://shopping.yahooapis.jp/ShoppingWebService/V3/itemSearch"

def search_by_jan_v3(client_id: str, jan: str, hits: int = 1):
    params = {
        "appid": client_id,
        "jan_code": jan,
        "results": hits,
        "output": "json"
    }
    # GET リクエスト実行
    resp = requests.get(API_ENDPOINT, params=params, timeout=10)
    resp.raise_for_status()
    return resp.json()

def parse_v3_response(resp_json):
    # v3 では “hits” 配列に複数ヒットが入る形式
    hits = resp_json.get("hits")
    if not hits:
        return None
    first = hits[0]
    # v3 のレスポンス例ではキー名が “name”, “price”, “url” など
    return {
        "name": first.get("name"),
        "price": first.get("price"),
        "url": first.get("url")
    }

if __name__ == "__main__":
    try:
        print(f"🔍 検索対象の JAN コード: {JAN_CODE}")
        resp_json = search_by_jan_v3(CLIENT_ID, JAN_CODE, hits=1)
        item = parse_v3_response(resp_json)
        if item:
            print("✅ 商品が見つかりました")
            print("   名前:", item["name"])
            print("   価格:", item["price"])
            print("   URL:", item["url"])
        else:
            print("❌ 該当商品なし")
    except requests.exceptions.HTTPError as e:
        print("HTTP エラー:", e)
        print("レスポンス内容:", e.response.text)
    except Exception as e:
        print("予期せぬエラー:", e)


🔍 検索対象の JAN コード: 622356249829
HTTP エラー: 400 Client Error: Bad Request for url: https://shopping.yahooapis.jp/ShoppingWebService/V3/itemSearch?appid=dj00aiZpPXlkOGd5bDlUcTlWRyZzPWNvbnN1bWVyc2VjcmV0Jng9MmE-&jan_code=622356249829&results=1&output=json
レスポンス内容: {"Error":{"Code":500,"Message":"\"Request failed with status code 400\""}}


In [8]:
# Authorizationエンドポイントへリダイレクト
client_id = "dj00aiZpPXlkOGd5bDlUcTlWRyZzPWNvbnN1bWVyc2VjcmV0Jng9MmE-"
url = 'https://auth.login.yahoo.co.jp/yconnect/v2/authorization'
params = {}
params ['response_type'] = 'code'
params ['client_id'] = client_id
params ['redirect_uri'] = 'https://www.yahoo.co.jp/'
params ['scope'] = 'openid'
response = requests.get(url, params)
print(response.url)


https://auth.login.yahoo.co.jp/yconnect/v2/authorization?response_type=code&client_id=dj00aiZpPXlkOGd5bDlUcTlWRyZzPWNvbnN1bWVyc2VjcmV0Jng9MmE-&redirect_uri=https%3A%2F%2Fwww.yahoo.co.jp%2F&scope=openid


In [10]:
import requests

API_URL = "https://shopping.yahooapis.jp/ShoppingWebService/V3/itemSearch"
APP_ID = "dj00aiZpPXlkOGd5bDlUcTlWRyZzPWNvbnN1bWVyc2VjcmV0Jng9MmE-"
JAN_CODE = "622356249829"

params = {
    "appid": APP_ID,
    "jan_code": JAN_CODE,
    "results": 3
}

res = requests.get(API_URL, params=params)
print(res.status_code)  # ← 200ならOK
print(res.json())


400
{'Error': {'Code': 500, 'Message': '"Request failed with status code 400"'}}


In [15]:
import requests

API_URL = "https://shopping.yahooapis.jp/ShoppingWebService/V3/itemSearch"
APP_ID = "dj00aiZpPXlkOGd5bDlUcTlWRyZzPWNvbnN1bWVyc2VjcmV0Jng9MmE-"

for jan in ["4901301026218", "4549077330020", "4960999918490"]:
    params = {"appid": APP_ID, "jan_code": jan, "results": 3}
    res = requests.get(API_URL, params=params)
    data = res.json()
    print("🔍 JAN:", jan)
    print("ヒット件数:", data.get("totalResultsAvailable", 0))
    if data.get("hits"):
        for item in data["hits"]:
            print("　商品名:", item.get("name"))
            print("　価格:", item.get("price"))
            print("　URL:", item.get("url"))
            print("　JAN:", item.get("janCode"))
            print("──────────────")
    else:
        print("　❌ 見つかりませんでした")


🔍 JAN: 4901301026218
ヒット件数: 0
　❌ 見つかりませんでした
🔍 JAN: 4549077330020
ヒット件数: 0
　❌ 見つかりませんでした
🔍 JAN: 4960999918490
ヒット件数: 0
　❌ 見つかりませんでした


In [3]:
import requests
import json

API_URL = "https://shopping.yahooapis.jp/ShoppingWebService/V3/itemSearch"
APP_ID = "dj00aiZpPXlkOGd5bDlUcTlWRyZzPWNvbnN1bWVyc2VjcmV0Jng9MmE-"  # ← あなたのClient ID

params = {
    "appid": APP_ID,
    "query": "お弁当袋",      # 🔍 検索キーワード
    "results": 10,           # 取得件数（最大50件）
    "sort": "-score",        # 並び順：おすすめ順
    "in_stock": "true",      # 在庫ありのみ
    "condition": "new"       # 新品のみ
}

response = requests.get(API_URL, params=params)
data = response.json()

print("🔍 検索ヒット数:", data.get("totalResultsAvailable", 0))
print("────────────────────────────")

for hit in data.get("hits", []):
    print("🛍 商品名:", hit.get("name"))
    print("💰 価格:", hit.get("price"), "円")
    print("🏪 ストア:", hit.get("seller", {}).get("name"))
    print("🔗 URL:", hit.get("url"))
    print("📦 在庫:", "あり" if hit.get("inStock") else "なし")
    print("──────────────")


🔍 検索ヒット数: 52007
────────────────────────────
🛍 商品名: 弁当袋 ランチバッグ お弁当袋 弁当箱 保温 保冷 小さめ コンパクトファスナー付き 撥水加工 軽量 シンプルデザイン 通勤 通学 メンズ レディース おしゃれ
💰 価格: 1289 円
🏪 ストア: Dog Fight
🔗 URL: https://store.shopping.yahoo.co.jp/dog-fight/g0-wle3-ec3x-y.html
📦 在庫: あり
──────────────
🛍 商品名: お弁当袋 保冷 保温 巾着ポーチ ランチバッグ 撥水 洗える ブランド wpc かわいい おしゃれ ギフト
💰 価格: 1650 円
🏪 ストア: 神戸岡本Kiitos
🔗 URL: https://store.shopping.yahoo.co.jp/kiitos-web/wpc-002.html
📦 在庫: あり
──────────────
🛍 商品名: ランチバッグ 保冷 保温 大きめ メンズ レディース 大容量 保冷 保温 エコ おしゃれ トート お弁当袋 サイドポケット ランチ バック バッグ
💰 価格: 780 円
🏪 ストア: ddice
🔗 URL: https://store.shopping.yahoo.co.jp/confianceshop/lunchbag1.html
📦 在庫: あり
──────────────
🛍 商品名: 【10%オフクーポン〜9日0時】ランチバッグ 全5色 保冷 保温 大きめ 縦長 お弁当袋 くすみカラー キッズ ランチバッグ 子ども ランチバッグ シンプル お弁当 遠足
💰 価格: 1980 円
🏪 ストア: atRise Yahoo!店
🔗 URL: https://store.shopping.yahoo.co.jp/risecreation/r0598.html
📦 在庫: あり
──────────────
🛍 商品名: ランチバッグ 縦長 保冷バッグ お弁当袋 保温 小さめ がま口 マジックテープ マチ 広い 子供 大人 防水
💰 価格: 550 円
🏪 ストア: リトルステップ
🔗 URL: https://store.shopping.yahoo.co.jp/littleste

In [ ]:
import requests

# 🌸 Yahoo!ショッピング 商品検索API（V3）
API_URL = "https://shopping.yahooapis.jp/ShoppingWebService/V3/itemSearch"
APP_ID = "dj00aiZpPXlkOGd5bDlUcTlWRyZzPWNvbnN1bWVyc2VjcmV0Jng9MmE-"  # あなたのClient ID

# 🌸 検索条件
SEARCH_CONDITION = {
    "seller_id": "hands-net",   # ストアID（ハンズネット）
    "price_from": 1000,         # 最低価格
    "price_to": 4000,           # 最高価格
    "results": 20,              # 取得件数
    "sort": "-score",           # 並び順（おすすめ順＝ランキング上位）
    "in_stock": "true",         # 在庫ありのみ
    "condition": "new"          # 新品のみ
}

# 🌷 条件を先頭に表示
print("🔍 検索条件")
print("────────────────────────────")
print(f"🏪 ストアID：{SEARCH_CONDITION['seller_id']}")
print(f"💰 価格範囲：{SEARCH_CONDITION['price_from']}円〜{SEARCH_CONDITION['price_to']}円")
print(f"📊 並び順　：おすすめ順（-score）")
print(f"📦 取得件数：{SEARCH_CONDITION['results']}件")
print("────────────────────────────\n")

# 🌸 APIリクエスト
params = {"appid": APP_ID, **SEARCH_CONDITION}
response = requests.get(API_URL, params=params)
data = response.json()

# 🌸 結果出力
total = data.get("totalResultsAvailable", 0)
print(f"📦 ヒット件数：{total}")
print("────────────────────────────")

hits = data.get("hits", [])
if not hits:
    print("❌ 条件に合う商品が見つかりませんでした。")
else:
    for i, hit in enumerate(hits, 1):
        print(f"#{i:02d} 🛍 {hit.get('name')}")
        print(f"💴 価格：{hit.get('price')}円")
        print(f"🏪 ストア：{hit.get('seller', {}).get('name')}")
        print(f"📦 JANコード：{hit.get('janCode') or '（未登録）'}")
        print(f"🔗 URL：{hit.get('url')}")
        print("──────────────")


In [33]:
import requests
import pandas as pd
import time

# 🌸 Yahoo!ショッピング 商品検索API（V3）
API_URL = "https://shopping.yahooapis.jp/ShoppingWebService/V3/itemSearch"
APP_ID = "dj00aiZpPXlkOGd5bDlUcTlWRyZzPWNvbnN1bWVyc2VjcmV0Jng9MmE-"  # あなたのClient ID

# 🌷 検索条件
QUERY = ""
PRICE_MIN = 500
PRICE_MAX = 5000
RESULTS_PER_CALL = 50
TOTAL_ITEMS = 1000
CALLS = TOTAL_ITEMS // RESULTS_PER_CALL

# 🌸 商品情報を格納するリスト
all_rows = []

print("🔍 Yahoo!ショッピングAPIで雑貨を取得中...")

for i in range(CALLS):
    start = i * RESULTS_PER_CALL + 1
    params = {
        "appid": APP_ID,
        "query": QUERY,
        "price_from": PRICE_MIN,
        "price_to": PRICE_MAX,
        "results": RESULTS_PER_CALL,
        "start": start,
        "sort": "-sold",           # おすすめ順
        "in_stock": "true",
        "condition": "new"
    }

    try:
        response = requests.get(API_URL, params=params, timeout=10)
        data = response.json()
        hits = data.get("hits", [])

        for h in hits:
            jan = h.get("janCode") or ""
            price = h.get("price") or ""
            name = h.get("name") or ""
            store = h.get("sellerId") or ""
            all_rows.append([store, jan, price, name])

        print(f"✅ {start}件目〜{start + RESULTS_PER_CALL - 1}件目まで取得完了")
        time.sleep(1)

    except Exception as e:
        print(f"⚠️ エラー（{start}件目〜）: {e}")
        time.sleep(3)

# 🌼 DataFrameに変換して保存
df = pd.DataFrame(all_rows, columns=["ストアID", "JANコード", "価格", "商品名"])
output_path = "雑貨_1000件リスト.xlsx"
df.to_excel(output_path, index=False)

print("\n🎉 完了！")
print(f"📦 取得件数: {len(df)} 件")
print(f"📁 保存先: {output_path}")


🔍 Yahoo!ショッピングAPIで雑貨を取得中...
✅ 1件目〜50件目まで取得完了
✅ 51件目〜100件目まで取得完了
✅ 101件目〜150件目まで取得完了
✅ 151件目〜200件目まで取得完了
✅ 201件目〜250件目まで取得完了
✅ 251件目〜300件目まで取得完了
✅ 301件目〜350件目まで取得完了
✅ 351件目〜400件目まで取得完了
✅ 401件目〜450件目まで取得完了
✅ 451件目〜500件目まで取得完了
✅ 501件目〜550件目まで取得完了
✅ 551件目〜600件目まで取得完了
✅ 601件目〜650件目まで取得完了
✅ 651件目〜700件目まで取得完了
✅ 701件目〜750件目まで取得完了
✅ 751件目〜800件目まで取得完了
✅ 801件目〜850件目まで取得完了
✅ 851件目〜900件目まで取得完了
✅ 901件目〜950件目まで取得完了
✅ 951件目〜1000件目まで取得完了

🎉 完了！
📦 取得件数: 950 件
📁 保存先: 雑貨_1000件リスト.xlsx


In [46]:
import requests
import time

# 🌸 Yahoo!ショッピング API設定
API_URL = "https://shopping.yahooapis.jp/ShoppingWebService/V3/itemSearch"
APP_ID = "dj00aiZpPXlkOGd5bDlUcTlWRyZzPWNvbnN1bWVyc2VjcmV0Jng9MmE-"  # あなたのClient ID

# 🌸 検索条件
SEARCH_CONDITION = {
    "seller_id": "hands-net",   # ストアID（例：ハンズネット）
    "price_from": 1000,         # 最低価格
    "price_to": 4000,           # 最高価格
    "results": 50,              # 1回あたりの取得件数（最大50）
    "sort": "-sold",           # 並び順（おすすめ順）
    "in_stock": "true",         # 在庫あり
    "condition": "new"          # 新品のみ
}

# 🌼 呼び出し回数を2回（合計100件）
CALLS = 2

# 🌷 条件を先頭に表示
print("🔍 検索条件")
print("────────────────────────────")
print(f"🏪 ストアID：{SEARCH_CONDITION['seller_id']}")
print(f"💰 価格範囲：{SEARCH_CONDITION['price_from']}円〜{SEARCH_CONDITION['price_to']}円")
print(f"📊 並び順　：おすすめ順（-score）")
print(f"📦 取得件数：50件 × {CALLS}回 ＝ 合計100件")
print("────────────────────────────\n")

# 🌸 APIリクエスト＆出力
total_count = 0

for i in range(CALLS):
    start = i * SEARCH_CONDITION["results"] + 1  # 開始位置（1, 51）
    params = {"appid": APP_ID, "start": start, **SEARCH_CONDITION}

    try:
        response = requests.get(API_URL, params=params, timeout=10)
        data = response.json()
        hits = data.get("hits", [])

        if not hits:
            print(f"⚠️ {start}〜{start + 49}件目：ヒットなし\n")
            continue

        print(f"✅ {start}〜{start + 49}件目を取得中...\n")

        for j, hit in enumerate(hits, 1):
            total_count += 1
            print(f"#{total_count:03d} 🛍 {hit.get('name')}")
            print(f"💴 価格：{hit.get('price')}円")
            print(f"🏪 ストア：{hit.get('seller', {}).get('name')}")
            print(f"📦 JANコード：{hit.get('janCode') or '（未登録）'}")
            print(f"🔗 URL：{hit.get('url')}")
            print("──────────────")

        time.sleep(1)  # Yahoo!APIに優しい待機

    except Exception as e:
        print(f"⚠️ エラー発生（{start}件目〜）: {e}")
        time.sleep(3)

# 🌸 結果まとめ
print("\n🎉 完了！")
print(f"📦 合計取得商品数：{total_count} 件")


🔍 検索条件
────────────────────────────
🏪 ストアID：hands-net
💰 価格範囲：1000円〜4000円
📊 並び順　：おすすめ順（-score）
📦 取得件数：50件 × 2回 ＝ 合計100件
────────────────────────────

✅ 1〜50件目を取得中...

#001 🛍 グリーンベル　匠の技　ステンレス製キャッチャー爪切り　L　G-1112 ハンズ
💴 価格：2310円
🏪 ストア：ハンズYahoo!ショッピング店
📦 JANコード：4972525052672
🔗 URL：https://store.shopping.yahoo.co.jp/hands-net/4972525052672.html
──────────────
#002 🛍 ハンズオリジナル　muqna（ムクナ）　化粧水　しっとり　400mL ハンズ
💴 価格：1480円
🏪 ストア：ハンズYahoo!ショッピング店
📦 JANコード：4549917232129
🔗 URL：https://store.shopping.yahoo.co.jp/hands-net/4549917232129.html
──────────────
#003 🛍 【2026年版・壁掛】　ほぼ日　ホワイトボードカレンダー　フルサイズ ハンズ
💴 価格：3080円
🏪 ストア：ハンズYahoo!ショッピング店
📦 JANコード：4573630261364
🔗 URL：https://store.shopping.yahoo.co.jp/hands-net/4573630261364.html
──────────────
#004 🛍 【お買い得】　共栄製茶　KFK　ドリップコーヒーバラエティパック　30袋 ハンズ
💴 価格：1382円
🏪 ストア：ハンズYahoo!ショッピング店
📦 JANコード：4904021120050
🔗 URL：https://store.shopping.yahoo.co.jp/hands-net/4904021120050.html
──────────────
#005 🛍 【お買い得】　ハンズオリジナル　muqna（ムクナ）　化粧水　しっとり　400ml×2　特別セット　【ネットストア限定】 ハンズ
💴 価格：26

In [48]:
import requests
import time

API_URL = "https://shopping.yahooapis.jp/ShoppingWebService/V3/itemSearch"
APP_ID = "dj00aiZpPXlkOGd5bDlUcTlWRyZzPWNvbnN1bWVyc2VjcmV0Jng9MmE-"

store_ids = set()

for i in range(10):  # 試しに10ページ分
    start = i * 50 + 1  # ページ先頭位置
    params = {
        "appid": APP_ID,
        "query": "スマートフォン",  # キーワードは必須
        "results": 50,             # 最大50件
        "start": start,
        "sort": "-score",          # 人気順
        "availability": "1",       # 文字列で指定（整数だと400エラーになる場合あり）
        "condition": "new"         # これも "new" / "used" の文字列指定
    }

    try:
        response = requests.get(API_URL, params=params, timeout=10)
        response.raise_for_status()
        data = response.json()

        hits = data.get("hits", [])
        if not hits:
            print(f"{i+1}ページ目にデータがありません。終了します。")
            break

        for h in hits:
            seller = h.get("seller", {})
            store = seller.get("sellerId")
            if store:
                store_ids.add(store)

        print(f"{i+1}ページ目完了。現在の店舗数: {len(store_ids)}")

        time.sleep(0.5)

    except requests.exceptions.RequestException as e:
        print(f"{i+1}ページ目でリクエストエラー: {e}")
        print(f"レスポンス: {response.text if 'response' in locals() else 'なし'}")
        break
    except Exception as e:
        print(f"予期せぬエラー: {e}")
        break

print(f"\n最終的に取得した店舗数: {len(store_ids)}")
print(store_ids)


1ページ目完了。現在の店舗数: 12
2ページ目完了。現在の店舗数: 28
3ページ目完了。現在の店舗数: 40
4ページ目完了。現在の店舗数: 54
5ページ目完了。現在の店舗数: 69
6ページ目完了。現在の店舗数: 86
7ページ目完了。現在の店舗数: 105
8ページ目完了。現在の店舗数: 119
9ページ目完了。現在の店舗数: 133
10ページ目完了。現在の店舗数: 146

最終的に取得した店舗数: 146
{'enmo-do', 'densidonya', 'alpen-group', 'n-style', 'akaikumasan', 'life-aozora-shop', 'tamatama2019', 'topwood', 'light-hikari', 'hanaro', 'only1-led', 'niconicodo', 'eccurrent', 'minesto-store', 'straight-toolcompany', 'telaffy', 'nyflood', 'murauchi3', 'initial-k', 'tk-retail2-fb', 'keiyo-officialshop', 'joshin-cddvd', 'ankerdirect', 'skydragon', 'joshin', 'sesame-store', 'webike', 'aprice', 'moto-zoa', 'mobilestation', 'yumeyakata', 'gbft-online', 'soft99e-mono', 'basecamp8', 'komo-ecy', 'gifteeegift', 'shizukawill', 'spices', 'adoingplus-shopping', 'softbank-selection', 'garmin-official', 'panda-mobile', 'dplus', 'live2', 'toysrus-babierus', 'bestone1', 'fastonline', 'ropping', 'norton', 'mkcorporation8', 'hcvalor', 'yunoki', 'ad-hitshop', 'inst', 'himaraya', 'warawara-st

In [2]:
import requests
import pandas as pd
import time

# 🌸 Yahoo!ショッピング API設定
API_URL = "https://shopping.yahooapis.jp/ShoppingWebService/V3/itemSearch"
APP_ID = "dj00aiZpPXlkOGd5bDlUcTlWRyZzPWNvbnN1bWVyc2VjcmV0Jng9MmE-"  # ← 有効なAPIキーを使用してください

rows = []
MAX_PAGES = 40          # 50件 × 40回 = 2000件
RESULTS_PER_PAGE = 50   # Yahoo!の上限

for i in range(MAX_PAGES):
    start = i * RESULTS_PER_PAGE + 1  # ページ開始位置（1, 51, 101, ...）
    params = {
        "appid": APP_ID,
        "seller_id": "lohaco",     # 🌸 LOHACO公式
        "price_from": 1000,
        "price_to": 10000,
        "results": RESULTS_PER_PAGE,
        "start": start,
        "sort": "-score",          # おすすめ順
        "in_stock": "true",
        "condition": "new"
    }

    try:
        response = requests.get(API_URL, params=params, timeout=10)
        response.raise_for_status()
        data = response.json()
        hits = data.get("hits", [])

        if not hits:
            print(f"{i+1}ページ目にデータがありません。終了します。")
            break

        for h in hits:
            jan = h.get("jan") or ""
            price = h.get("price") or ""
            name = h.get("name") or ""
            rows.append([jan, price, name])

        print(f"{i+1}ページ目完了 ✅ 現在 {len(rows)} 件取得。")
        time.sleep(0.5)  # API制限対策

    except requests.exceptions.RequestException as e:
        print(f"{i+1}ページ目でリクエストエラー: {e}")
        time.sleep(1)
    except ValueError as e:
        print(f"{i+1}ページ目でJSON解析エラー: {e}")
        time.sleep(1)
    except Exception as e:
        print(f"{i+1}ページ目で予期せぬエラー: {e}")
        time.sleep(1)

# 🌸 Excelに保存
df = pd.DataFrame(rows, columns=["JANコード", "価格", "商品名"])
output_path = "lohaco_top2000.xlsx"
df.to_excel(output_path, index=False)

print("✅ データ取得・保存完了！")
print(f"📁 保存先: {output_path}")


1ページ目にデータがありません。終了します。
✅ データ取得・保存完了！
📁 保存先: lohaco_top2000.xlsx


In [51]:
import requests
import pandas as pd
import time

# Yahoo! API設定
API_URL = "https://shopping.yahooapis.jp/ShoppingWebService/V3/itemSearch"
APP_ID = "dj00aiZpPXlkOGd5bDlUcTlWRyZzPWNvbnN1bWVyc2VjcmV0Jng9MmE-"

# Excelファイル読み込み
df = pd.read_excel(r"C:\Users\taku5\Desktop\テスト用.xlsx")

# Yahoo最安価格列を「価格」の次に挿入
insert_position = df.columns.get_loc("価格") + 1
df.insert(insert_position, "Yahoo最安価格", None)

# JANコードでのみ検索
for idx, row in df.iterrows():
    jan = str(row["JANコード"]).strip()

    # JANが空ならスキップ
    if not jan or jan == "nan":
        print(f"{idx+1}/{len(df)}件目: JANなし → スキップ")
        continue

    params = {
        "appid": APP_ID,
        "jan_code": jan,
        "results": 1,         # 最安1件だけ
        "sort": "+price",     # 価格昇順
        "in_stock": "true",
        "condition": "new"
    }

    try:
        response = requests.get(API_URL, params=params, timeout=10)
        response.raise_for_status()
        data = response.json()
        hits = data.get("hits", [])
        if hits:
            lowest_price = hits[0].get("price")
            df.at[idx, "Yahoo最安価格"] = lowest_price
            print(f"{idx+1}/{len(df)}件目: JAN={jan} → 最安 {lowest_price}円")
        else:
            print(f"{idx+1}/{len(df)}件目: JAN={jan} → 該当なし")

        time.sleep(0.5)  # API制限対策

    except Exception as e:
        print(f"{idx+1}件目でエラー: {e}")
        time.sleep(1)

# 結果を保存
output_path = "handsnet_with_min_price_inserted.xlsx"
df.to_excel(output_path, index=False)

print("✅ 完了！『Yahoo最安価格』を2列目（価格の隣）に挿入しました。")
print(f"📁 保存先: {output_path}")


"""
ファイルから最安値いれる
"""


1/9件目: JAN=4573630261364 → 最安 3080円
2/9件目: JAN=4972525052672 → 最安 2310円
3/9件目: JAN=4589860481922 → 最安 2200円
4/9件目: JAN=4573630261388 → 最安 1870円
5/9件目: JAN=4939871206517 → 最安 2178円
6/9件目: JAN=4549917232129 → 最安 1480円
7/9件目: JAN=4573630240277 → 最安 3410円
8/9件目: JAN=4902805313971 → 最安 2750円
9/9件目: JAN=2431000064736 → 最安 2664円
✅ 完了！『Yahoo最安価格』を2列目（価格の隣）に挿入しました。
📁 保存先: handsnet_with_min_price_inserted.xlsx


In [10]:
import requests
import pandas as pd
import time

# 🌸 Yahoo!ショッピング 商品検索API（V3）
API_URL = "https://shopping.yahooapis.jp/ShoppingWebService/V3/itemSearch"
APP_ID = "dj00aiZpPXlkOGd5bDlUcTlWRyZzPWNvbnN1bWVyc2VjcmV0Jng9MmE-"  # あなたのClient ID

# 🌷 検索条件（ロハコ限定）
SELLER_ID = "lohaco"        # ロハコYahoo!店を指定
RESULTS_PER_CALL = 50
TOTAL_ITEMS = 100           # 取得したい件数
CALLS = TOTAL_ITEMS // RESULTS_PER_CALL

# 🌷 価格帯（必要に応じて変更可能）
PRICE_MIN = 1000
PRICE_MAX = 10000

# 🌸 商品情報を格納するリスト
all_rows = []

print("🔍 Yahoo!ショッピングAPIでロハコ商品を取得中...")

for i in range(CALLS):
    start = i * RESULTS_PER_CALL + 1
    params = {
            "seller_id": "hands-net",   # ストアID（例：ハンズネット）
            "price_from": 1000,         # 最低価格
            "price_to": 4000,           # 最高価格
            "results": 50,              # 1回あたりの取得件数（最大50）
            "sort": "-sold",           # 並び順（おすすめ順）
            "in_stock": "true",         # 在庫あり
            "condition": "new"          # 新品のみ
    }

    try:
        response = requests.get(API_URL, params=params, timeout=10)
        response.raise_for_status()
        data = response.json()
        hits = data.get("hits", [])

        if not hits:
            print(f"⚠️ {i+1}ページ目：商品が見つかりません。終了します。")
            break

        for h in hits:
            jan = h.get("jan") or ""
            price = h.get("price") or ""
            name = h.get("name") or ""
            store = h.get("seller", {}).get("name") or ""
            url = h.get("url") or ""
            all_rows.append([store, jan, price, name, url])

        print(f"✅ {start}件目〜{start + RESULTS_PER_CALL - 1}件目まで取得完了")
        time.sleep(1)

    except Exception as e:
        print(f"⚠️ エラー（{start}件目〜）: {e}")
        time.sleep(3)

# 🌼 DataFrameに変換して保存
df = pd.DataFrame(all_rows, columns=["ストア名", "JANコード", "価格", "商品名", "商品URL"])
output_path = "ロハコ商品リスト_100件.xlsx"
df.to_excel(output_path, index=False)

print("\n🎉 完了！")
print(f"📦 取得件数: {len(df)} 件")
print(f"📁 保存先: {output_path}")


🔍 Yahoo!ショッピングAPIでロハコ商品を取得中...
⚠️ エラー（1件目〜）: 401 Client Error: Unauthorized for url: https://shopping.yahooapis.jp/ShoppingWebService/V3/itemSearch?seller_id=hands-net&price_from=1000&price_to=4000&results=50&sort=-sold&in_stock=true&condition=new
⚠️ エラー（51件目〜）: 401 Client Error: Unauthorized for url: https://shopping.yahooapis.jp/ShoppingWebService/V3/itemSearch?seller_id=hands-net&price_from=1000&price_to=4000&results=50&sort=-sold&in_stock=true&condition=new

🎉 完了！
📦 取得件数: 0 件
📁 保存先: ロハコ商品リスト_100件.xlsx


In [18]:
import requests
import pandas as pd
import time

# 🌸 Yahoo!ショッピング 商品検索API（V3）
API_URL = "https://shopping.yahooapis.jp/ShoppingWebService/V3/itemSearch"
APP_ID = "dj00aiZpPXlkOGd5bDlUcTlWRyZzPWNvbnN1bWVyc2VjcmV0Jng9MmE-"  # あなたのClient ID

# 🌷 検索設定
SELLER_ID = "ishibashi"   # 東急ハンズYahoo!店
RESULTS_PER_CALL = 50      # 1回で取得できる最大件数
TOTAL_ITEMS = 2000         # 目標取得件数
CALLS = TOTAL_ITEMS // RESULTS_PER_CALL

all_rows = []

print("🔍 Yahoo!ショッピングAPIから商品情報を取得中...")

for i in range(CALLS):
    start = i * RESULTS_PER_CALL + 1
    params = {
        "appid": APP_ID,
        "seller_id": SELLER_ID,
        "price_from": 100,
        "price_to": 100000,
        "results": RESULTS_PER_CALL,
        "start": start,
        "sort": "-score",      # おすすめ順
        "in_stock": "true",
        "condition": "new"
    }

    try:
        response = requests.get(API_URL, params=params, timeout=10)
        data = response.json()
        hits = data.get("hits", [])
        if not hits:
            print(f"⚠️ {i+1}ページ目に商品なし。")
            break

        for h in hits:
            jan = h.get("janCode") or ""
            price = h.get("price") or ""
            name = h.get("name") or ""
            url = h.get("url") or ""
            all_rows.append([jan, price, name, url])

        print(f"✅ {i+1}ページ目完了（{len(all_rows)}件）")

        time.sleep(0.5)  # 💡 API負荷軽減のため少し待機

    except Exception as e:
        print(f"❌ エラー: {e}")
        time.sleep(5)

# 🌼 DataFrameに変換してExcel出力
df = pd.DataFrame(all_rows, columns=["JANコード", "価格", "商品名", "商品URL"])
output_path = "handsnet_2000items.xlsx"
df.to_excel(output_path, index=False)

print("🎉 完了！Excelファイルを作成しました。")
print(f"📁 保存先: {output_path}")


🔍 Yahoo!ショッピングAPIから商品情報を取得中...
✅ 1ページ目完了（50件）
✅ 2ページ目完了（100件）
✅ 3ページ目完了（150件）
✅ 4ページ目完了（200件）
✅ 5ページ目完了（250件）
✅ 6ページ目完了（300件）
✅ 7ページ目完了（350件）
✅ 8ページ目完了（400件）
✅ 9ページ目完了（450件）
✅ 10ページ目完了（500件）
✅ 11ページ目完了（550件）
✅ 12ページ目完了（600件）
✅ 13ページ目完了（650件）
✅ 14ページ目完了（700件）
✅ 15ページ目完了（750件）
✅ 16ページ目完了（800件）
✅ 17ページ目完了（850件）
✅ 18ページ目完了（900件）
✅ 19ページ目完了（950件）
⚠️ 20ページ目に商品なし。
🎉 完了！Excelファイルを作成しました。
📁 保存先: handsnet_2000items.xlsx
